In [ ]:
import datetime
import csv
import sys
import itertools as it
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_context('talk')

In [ ]:
def str_hm(minutes):
    return f'{int(minutes//60)}h {int(minutes % 60)}m'

def str_t(minutes):
    return f'{int(minutes//60):02d}:{int(minutes % 60):02d}'

def int_or_none(s):
    return int(s) if s else None

def minutes(s):
    if not s:
        return None
    assert len(s) == 4
    return 60 * int(s[0:2]) + int(s[2:4])

def load_csv(path, columns):
    with open(path) as f:
        for record in csv.DictReader(f):
            assert record.keys() == {cname for _, _, cname in columns}
            try:
                yield {
                    cid: ctype(record[cname])
                    for cid, ctype, cname in columns
                }
            except ValueError as e:
                print(f'Error! could not parse {record}: {e}', file=sys.stderr)
                
def load_csv_dict(path, columns):
    return {date: list(values) for date, values in it.groupby(load_csv(path, columns), lambda x: x['date'])}

DAYS_COLUMNS = [
    ('date', str, 'Date'),
    ('wake_time', minutes, 'Wakeup time'),
    ('wake_mood', str, 'Wakeup mood'),
    ('bed_start', minutes, 'Bedtime start'),
    ('bed_down', minutes, 'Bedtime down'),
    ('bed_sleep', minutes, 'Bedtime sleep'),
    ('notes', str, 'Notes'),
]
NAPS_COLUMNS = [
    ('date', str, 'Date'),
    ('start', minutes, 'Nap time'),
    ('end', minutes, 'Wake time'),
    ('notes', str, 'Notes'),
]
NIGHT_WAKINGS_COLUMNS = [
    ('date', str, 'Date'),
    ('start', minutes, 'Wake time'),
    ('duration', int, 'Wake duration /mins'),
    ('response', str, 'Wake response'),
]

days = load_csv_dict('data/sleep/days.csv', DAYS_COLUMNS)
naps = load_csv_dict('data/sleep/naps.csv', NAPS_COLUMNS)
wakes = load_csv_dict('data/sleep/night_wakings.csv', NIGHT_WAKINGS_COLUMNS)

data = []
for date, day in days.items():
    assert len(day) == 1
    record = dict(day[0])
    record['naps'] = naps.get(date, [])
    record['wakes'] = wakes.get(date, [])
    data.append(record)

In [ ]:
records = []
for today, tomorrow in zip(data, data[1:-1]):
    awake_at_night = sum(w['duration'] for w in today['wakes'])
    records.append(dict(
        date=today['date'],
        sleep_at_night=24*60 - today["bed_sleep"] + tomorrow['wake_time'] - awake_at_night,
        awake_at_night=awake_at_night,
        sleep_in_day=sum(n['end'] - n['start'] for n in today['naps']),
        bedtime=today['bed_sleep'],
        waketime=tomorrow['wake_time'],
    ))

df = pd.DataFrame.from_dict(records)

one_week_ago = (datetime.datetime.now() - datetime.timedelta(days=7)).date().isoformat()
lw = df[one_week_ago <= df.date]
for title, d in [['Last 7 days', lw], ['Since records began', df]]:
    print(title)
    print(f'  Average sleep /night: {str_hm(d.sleep_at_night.mean())}')
    print(f'  Average awake /night: {d.awake_at_night.mean():.0f}m')
    print(f'  Average nap /day: {str_hm(d.sleep_in_day.mean())}')
    print()

In [ ]:
plt.figure(figsize=(14, 6))
xs = np.arange(len(df))
plt.bar(xs - .2, df.sleep_at_night, width=.4)
plt.bar(xs - .2, df.sleep_in_day, bottom=df.sleep_at_night, width=.4)
plt.bar(xs + .2, df.awake_at_night, width=.4)
plt.xticks(xs[::4], df.date[::4], rotation=25)
plt.legend(['Sleep at night', 'Sleep during day', 'Awake at night'], loc='upper left')
plt.gca().yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, _: f'{int(x)//60}h'))
plt.yticks(range(0, 16*60+1, 120))
plt.title('Sleep per day');

In [ ]:
ys = np.arange(len(df))
fig, (ax0, ax1) = plt.subplots(ncols=2, sharey=True, figsize=(10, 8))
# plt.title('Night sleep timing')
plt.suptitle('Night sleep timing')
ax0.barh(ys, 24*60-df.bedtime, align='center', color='gray')
ax1.barh(ys, df.waketime, align='center', color='gray')
ax0.invert_xaxis()
ax0.set_yticks(ys[::4])
ax0.set_yticklabels(df.date[::4])
ax1.get_yaxis().set_visible(False)
ax0.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, _: f'{str_t(24*60-x)}'))
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, _: f'{str_t(x)}'))
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
rng = data[-15:]
for today, tomorrow in zip(rng, rng[1:-1]):
    print(today['date'])
    sleep_total_incl = 24*60 - today["bed_sleep"] + tomorrow['wake_time']
    wakes_total = sum(w['duration'] for w in today['wakes'])
    naps_total = sum(n['end'] - n['start'] for n in today['naps'])

    print(f'  Total sleep time: {str_hm(sleep_total_incl - wakes_total + naps_total)}')
    print(f'  Night: {str_t(today["bed_sleep"])} - {str_t(tomorrow["wake_time"])} [{str_hm(sleep_total_incl)}]')
    print(f'  Time awake at night: {str_hm(wakes_total)}  ({len(today["wakes"])})')
    for wake in today["wakes"]:
        print(f'      {str_t(wake["start"])} [{str_hm(wake["duration"])}]')

    print(f'  Time napping in day: {str_hm(naps_total)}  ({len(today["naps"])})')
    for nap in today["naps"]:
        print(f'      {str_t(nap["start"])} [{str_hm(nap["end"] - nap["start"])}]')
    
    print()